In [1]:
import importlib
from sage.all_cmdline import *   # import sage library
from ctmc.metrics import *
from ctmc.generator import *

In [2]:
par = {k: 100, gamm: 0.00083, beta: 0.314, omegaA: 60, omegaB: 40}

In [7]:
(mining_throughput(pi) == total_throughput(pi)).subs(par)

0.09326036765671755 == 0.09326036765671757

In [5]:
pi.subs(par)

(0.4573006617709436, 0.19904063213177772, 0.14447224318265123, 0.09409979741795939, 0.028123225521950436, 0.057124387861918056, 0.014593817040464937, 0.0038497433507232873, 0.001395491721611256)